In [19]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import subprocess

In [42]:
curr_dir = !pwd
input_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/01.25.2018_domain_CV/per_fold/"
output_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/01.25.2018_domain_CV/"
#ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]
ligands = ["dna"]
#classifiers = ["RF", "SVM", "KNN", "ADA", "XGB"]
classifiers = ["XGB"]
#versions = ["10","10w","5","5w","3","3w"]
version = "10w"

#### Copy relevant files from gen-comp1

In [27]:
!scp -r -p anatf@gen-comp1:/Genomics/grid/users/anatf/ExAC/10.Prediction/pred_AUC_AUPRC/mediode_NegLigand_NoFilter/NoDown/01.25.2018_domain_CV/per_fold/* /home/anat/Research/ExAC/10.Prediction/pred_AUC_AUPRC/mediode_NegLigand_NoFilter/NoDown/01.25.2018_domain_CV/per_fold/.

dna_XGB_fold10_10w_auc.csv                    100%   25     0.0KB/s   00:00    
dna_XGB_fold10_10w_auprc.csv                  100%   26     0.0KB/s   00:00    
dna_XGB_fold1010w.csv                         100%  120KB 120.4KB/s   00:00    
dna_XGB_fold1_10w_auc.csv                     100%   26     0.0KB/s   00:00    
dna_XGB_fold1_10w_auprc.csv                   100%   26     0.0KB/s   00:00    
dna_XGB_fold110w.csv                          100%  139KB 138.8KB/s   00:00    
dna_XGB_fold2_10w_auc.csv                     100%   26     0.0KB/s   00:00    
dna_XGB_fold2_10w_auprc.csv                   100%   28     0.0KB/s   00:00    
dna_XGB_fold210w.csv                          100%  122KB 121.7KB/s   00:00    
dna_XGB_fold3_10w_auc.csv                     100%   26     0.0KB/s   00:00    
dna_XGB_fold3_10w_auprc.csv                   100%   27     0.0KB/s   00:00    
dna_XGB_fold310w.csv                          100%  118KB 118.4KB/s   00:00    
dna_XGB_fold4_10w_auc.csv               

In [48]:
for ligand in ligands:
    
    #Combine AUC folds
    for classifier in classifiers:
        auc_df = pd.DataFrame()
        auc_list = []
        for fold in range(1,11):
            auc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_auc.csv"
            curr_auc_df = pd.read_csv(auc_fold_filename, sep='\t', index_col=0)
            auc_list.append(float(curr_auc_df.iloc[0]))
        auc_df[classifier] = auc_list
        print ligand+" "+classifier+" AUC avg:"+str(np.mean(auc_list))
    #Write AUC table to file
    auc_df.to_csv(output_path+ligand+"_"+version+"_auc.csv", sep='\t')
    
    #Combine AUPRC folds
    for classifier in classifiers:
        auprc_df = pd.DataFrame()
        auprc_list = []
        for fold in range(1,11):
            auprc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_auprc.csv"
            curr_auprc_df = pd.read_csv(auprc_fold_filename, sep='\t', index_col=0)
            auprc_list.append(float(curr_auprc_df.iloc[0]))
        auprc_df[classifier] = auprc_list
        print ligand+" "+classifier+" AUPRC avg:"+str(np.mean(auprc_list))
    #Write AUPRC table to file
    auprc_df.to_csv(output_path+ligand+"_"+version+"_auprc.csv", sep='\t')
    
    #Combining probs results files
    probs_df = pd.DataFrame()
    for classifier in classifiers:
        for fold in range(1,11):
            probs_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+".csv"
            curr_probs_df = pd.read_csv(probs_filename, sep='\t', index_col=0)
            if (len(probs_df.columns) == 0):
                probs_df = curr_probs_df
            else:
                probs_df = probs_df.append(curr_probs_df, ignore_index=True)
    #Write probs table to file 
    probs_df.to_csv(output_path+ligand+"_"+version+".csv", sep='\t')

print "Finished ligand "+ligand 
            
        
        

dna XGB AUC avg:0.838831753004
dna XGB AUPRC avg:0.292736240131
Finished ligand dna
